In [3]:
import numpy as np
import pandas as pd
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

In [2]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [37]:
# load data
rawData = pd.read_csv("../input/train.csv").values
rawSubmission = pd.read_csv("../input/test.csv").values.astype("float32")

In [60]:
np.random.shuffle(rawData)

training, test = rawData[:32000,:], rawData[32000:,:]

# extract labels and features 
rawTrainY = training[:,0].astype("int")
trainY = np_utils.to_categorical(rawTrainY)

rawTestY = test[:,0].astype("int")
testY = np_utils.to_categorical(rawTestY)

trainRawX = training[:,1:].astype("float32")
trainNormX = trainRawX / 255.0
trainX = trainNormX.reshape( (32000,28,28,1) )

testRawX = test[:,1:].astype("float32")
testNormX = testRawX / 255.0
testX = testNormX.reshape( (10000,28,28,1) )

num_classes = trainY.shape[1]

In [53]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(28, 28, 1), border_mode='same', activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same', W_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', W_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [54]:
# Compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_7 (Convolution2D)  (None, 28, 28, 32)    320         convolution2d_input_6[0][0]      
____________________________________________________________________________________________________
dropout_3 (Dropout)              (None, 28, 28, 32)    0           convolution2d_7[0][0]            
____________________________________________________________________________________________________
convolution2d_8 (Convolution2D)  (None, 28, 28, 32)    9248        dropout_3[0][0]                  
____________________________________________________________________________________________________
maxpooling2d_2 (MaxPooling2D)    (None, 14, 14, 32)    0           convolution2d_8[0][0]            
___________________________________________________________________________________________

In [ ]:
model.fit(trainX, trainY, validation_data=(testX, testY), nb_epoch=epochs, batch_size=32)
# Final evaluation of the model
scores = model.evaluate(testX, testY, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))